In [1]:
# Import libraries

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrix
import datetime
import pickle
import statsmodels.regression.linear_model
import statsmodels.api as sm


# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
weather = pd.read_csv("../asset/weather.csv")
key = pd.read_csv("../asset/key.csv")
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")

# Success - Display the first record
print("weather  data : ", weather.shape)

import awesome_functions as cf

# 원본을 유지하기 위해서 카피
df_weather = weather.copy()
df_key = key.copy()
df_train = train.copy()
df_test = test.copy()

# heatmap을 위한 cmap은 이거 계속 쓰시면 됩니다.
cmap = sns.light_palette("orange", as_cmap=True)

weather  data :  (20517, 20)


# 원하는 스테이션 날씨 가져오기

In [2]:
train_preprocess = pickle.load(open("preprocess_date.pkl", "rb"))

In [3]:
train_preprocess

,date,store_nbr,item_nbr,units,log1p,date2,station_nbr,preciptotal_flag,depart_flag,weekday,is_weekend,is_holiday,is_holiday_weekday,is_holiday_weekend,day,month,year,holiday_name,around_BlackFriday
0,2012-01-01,1,9,29,3.401197,2012-01-01,1,0.0,0.0,6,1,1,0,1,1,1,2012,NewYearsDay,Else
1,2012-01-01,1,28,2,1.098612,2012-01-01,1,0.0,0.0,6,1,1,0,1,1,1,2012,NewYearsDay,Else
2,2012-01-01,1,40,0,0.000000,2012-01-01,1,0.0,0.0,6,1,1,0,1,1,1,2012,NewYearsDay,Else
3,2012-01-01,1,47,0,0.000000,2012-01-01,1,0.0,0.0,6,1,1,0,1,1,1,2012,NewYearsDay,Else
4,2012-01-01,1,51,1,0.693147,2012-01-01,1,0.0,0.0,6,1,1,0,1,1,1,2012,NewYearsDay,Else
5,2012-01-01,1,89,0,0.000000,2012-01-01,1,0.0,0.0,6,1,1,0,1,1,1,2012,NewYearsDay,Else
6,2012-01-01,1,93,0,0.000000,2012-01-01,1,0.0,0.0,6,1,1,0,1,1,1,2012,NewYearsDay,Else
7,2012-01-01,1,99,0,0.000000,2012-01-01,1,0.0,0.0,6,1,1,0,1,1,1,2012,NewYearsDay,Else
8,2012-01-02,1,9,60,4.110874,2012-01-02,1,0.0,0.0,0,0,1,1,0,2,1,2012,,Else
9,2012-01-02,1,28,5,1.791759,2012-01-02,1,0.0,0.0,0,0,1,1,0,2,1,2012,,Else


In [4]:
df_train_key= df_train.merge(df_key, on=["store_nbr"], how="inner")

In [5]:
df_train_key.head()

,date,store_nbr,item_nbr,units,station_nbr
0,2012-01-01,1,1,0,1
1,2012-01-01,1,2,0,1
2,2012-01-01,1,3,0,1
3,2012-01-01,1,4,0,1
4,2012-01-01,1,5,0,1


In [6]:
df_total = train_preprocess.merge(df_weather, on=["date","station_nbr"],how="inner")

In [7]:
df_total

,date,store_nbr,item_nbr,units,log1p,date2,station_nbr,preciptotal_flag,depart_flag,weekday,...,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,2012-01-01,1,9,29,3.401197,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2012-01-01,1,28,2,1.098612,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
2,2012-01-01,1,40,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
3,2012-01-01,1,47,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
4,2012-01-01,1,51,1,0.693147,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
5,2012-01-01,1,89,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
6,2012-01-01,1,93,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
7,2012-01-01,1,99,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
8,2012-01-02,1,9,60,4.110874,2012-01-02,1,0.0,0.0,0,...,-,-,,M,0.01,29.44,29.62,9.8,24,10.3
9,2012-01-02,1,28,5,1.791759,2012-01-02,1,0.0,0.0,0,...,-,-,,M,0.01,29.44,29.62,9.8,24,10.3


In [8]:
df_total_5 = df_total[df_total["store_nbr"]!=35]

In [9]:
df_total_5

,date,store_nbr,item_nbr,units,log1p,date2,station_nbr,preciptotal_flag,depart_flag,weekday,...,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,2012-01-01,1,9,29,3.401197,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2012-01-01,1,28,2,1.098612,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
2,2012-01-01,1,40,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
3,2012-01-01,1,47,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
4,2012-01-01,1,51,1,0.693147,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
5,2012-01-01,1,89,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
6,2012-01-01,1,93,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
7,2012-01-01,1,99,0,0.000000,2012-01-01,1,0.0,0.0,6,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
8,2012-01-02,1,9,60,4.110874,2012-01-02,1,0.0,0.0,0,...,-,-,,M,0.01,29.44,29.62,9.8,24,10.3
9,2012-01-02,1,28,5,1.791759,2012-01-02,1,0.0,0.0,0,...,-,-,,M,0.01,29.44,29.62,9.8,24,10.3


In [10]:
cf.isThereNoneData(df_total_5)

M Data over 60% : ['depart']
T Data over 60% : []
B Data over 60% : ['sunrise', 'sunset']


,Column,Row Count,Missing Data,M Data %,Trace Data,T Data %,Bar Data,B Data %
0,date,229230,0,0.00,0,0.00,0,0.00
1,store_nbr,229230,0,0.00,0,0.00,0,0.00
2,item_nbr,229230,0,0.00,0,0.00,0,0.00
3,units,229230,0,0.00,0,0.00,0,0.00
4,log1p,229230,0,0.00,0,0.00,0,0.00
5,date2,229230,0,0.00,0,0.00,0,0.00
6,station_nbr,229230,0,0.00,0,0.00,0,0.00
7,preciptotal_flag,229230,0,0.00,0,0.00,0,0.00
8,depart_flag,229230,0,0.00,0,0.00,0,0.00
9,weekday,229230,0,0.00,0,0.00,0,0.00


In [11]:
new_columns = list(weather.columns) + ["store_nbr","item_nbr","units"]
new_columns

['station_nbr',
 'date',
 'tmax',
 'tmin',
 'tavg',
 'depart',
 'dewpoint',
 'wetbulb',
 'heat',
 'cool',
 'sunrise',
 'sunset',
 'codesum',
 'snowfall',
 'preciptotal',
 'stnpressure',
 'sealevel',
 'resultspeed',
 'resultdir',
 'avgspeed',
 'store_nbr',
 'item_nbr',
 'units']

In [12]:
df_tot_reg = df_total_5[new_columns]
df_tot_reg

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,store_nbr,item_nbr,units
0,1,2012-01-01,52,31,42,M,36,40,23,0,...,M,0.05,29.78,29.92,3.6,20,4.6,1,9,29
1,1,2012-01-01,52,31,42,M,36,40,23,0,...,M,0.05,29.78,29.92,3.6,20,4.6,1,28,2
2,1,2012-01-01,52,31,42,M,36,40,23,0,...,M,0.05,29.78,29.92,3.6,20,4.6,1,40,0
3,1,2012-01-01,52,31,42,M,36,40,23,0,...,M,0.05,29.78,29.92,3.6,20,4.6,1,47,0
4,1,2012-01-01,52,31,42,M,36,40,23,0,...,M,0.05,29.78,29.92,3.6,20,4.6,1,51,1
5,1,2012-01-01,52,31,42,M,36,40,23,0,...,M,0.05,29.78,29.92,3.6,20,4.6,1,89,0
6,1,2012-01-01,52,31,42,M,36,40,23,0,...,M,0.05,29.78,29.92,3.6,20,4.6,1,93,0
7,1,2012-01-01,52,31,42,M,36,40,23,0,...,M,0.05,29.78,29.92,3.6,20,4.6,1,99,0
8,1,2012-01-02,50,31,41,M,26,35,24,0,...,M,0.01,29.44,29.62,9.8,24,10.3,1,9,60
9,1,2012-01-02,50,31,41,M,26,35,24,0,...,M,0.01,29.44,29.62,9.8,24,10.3,1,28,5


In [13]:
cf.remove_columns(df_tot_reg, ["date","depart","codesum","station_nbr","sunrise","sunset"])

제거한 컬럼명 : ['date', 'depart', 'codesum', 'station_nbr', 'sunrise', 'sunset']


,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,store_nbr,item_nbr,units
229225,82,59,71,60,63,0,6,M,0.00,29.74,M,0.3,11,0.2,39,111,3
229226,78,55,67,M,M,0,2,M,T,29.82,M,M,M,5.1,39,9,12
229227,78,55,67,M,M,0,2,M,T,29.82,M,M,M,5.1,39,36,3
229228,78,55,67,M,M,0,2,M,T,29.82,M,M,M,5.1,39,103,4
229229,78,55,67,M,M,0,2,M,T,29.82,M,M,M,5.1,39,111,2


In [14]:
df_tot_reg = cf.remove_m_row(df_tot_reg, df_tot_reg.columns)

원본 데이터프레임 length : 229230
제거하고 싶은 컬럼 : ['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed', 'store_nbr', 'item_nbr', 'units']
제거한 int 컬럼 : ['tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool']
제거한 float 컬럼 : ['snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']
제거한 후 데이터프레임 length : 134987


In [15]:
df_tot_reg.head()

,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,store_nbr,item_nbr,units
7432,50.0,34.0,42.0,25.0,35.0,23.0,0.0,0.0,0.0,29.13,30.52,11.4,32.0,11.3,2,5,191
7433,50.0,34.0,42.0,25.0,35.0,23.0,0.0,0.0,0.0,29.13,30.52,11.4,32.0,11.3,2,11,0
7434,50.0,34.0,42.0,25.0,35.0,23.0,0.0,0.0,0.0,29.13,30.52,11.4,32.0,11.3,2,44,215
7435,50.0,34.0,42.0,25.0,35.0,23.0,0.0,0.0,0.0,29.13,30.52,11.4,32.0,11.3,2,85,0
7436,50.0,34.0,42.0,25.0,35.0,23.0,0.0,0.0,0.0,29.13,30.52,11.4,32.0,11.3,2,93,0


In [16]:
y = ["units"]
x = list(df_tot_reg.columns)[:-1]

In [17]:
dfx0 = df_tot_reg[x]
dfy = df_tot_reg["units"]
dfX = sm.add_constant(dfx0) 

In [18]:
model = sm.OLS.from_formula("units ~ C(store_nbr):C(item_nbr) + tavg:C(item_nbr) + preciptotal:C(item_nbr) + 0", df_tot_reg)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     1449.
Date:                Mon, 02 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:14:47   Log-Likelihood:            -5.9341e+05
No. Observations:              134987   AIC:                         1.187e+06
Df Residuals:                  134655   BIC:                         1.191e+06
Df Model:                         331                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
C(stor

In [ ]:
dfy_new = result.predict(dfX)

plt.scatter(dfy, dfy_new)
plt.xlabel(u"real unit")
plt.ylabel(u"unit predict")
plt.show()

In [ ]:
#테스트 파일 합성
df_test_key= df_test.merge(df_key, on=["store_nbr"], how="inner")
df_test_key

In [ ]:
df_test_total = df_test_key.merge(df_weather, on=["date","station_nbr"],how="inner")

In [ ]:
df_test_total

In [ ]:
def getStoreList(df, item_nbr, msg=True):
    df_ = df.copy()
    df_ = df_[df_["item_nbr"] == item_nbr]
    df_ = df_[df_["units"] != 0]
    if msg:
        print("팔린 개수가 0이상인 row 개수 : " + str(len(df_)))
        print("총 팔린 개수 : " + str(df_["units"].sum()))
        print(str(item_nbr) + "번 아이템이 팔린 스토어 개수 : " + str(len(list(df_["store_nbr"].unique()))))
        print(str(item_nbr) + "번 아이템이 팔린 스토어 넘버 리스트 : " + str(list(df_["store_nbr"].unique())))
    return len(df_), str(df_["units"].sum()), list(df_["store_nbr"].unique())

In [ ]:
a,b,c = getStoreList(train_preprocess,5)

In [ ]:
df_item_5 = df_total[df_total["item_nbr"] ==5]

In [ ]:
c

In [ ]:
df_item_5.head()

In [ ]:
# 판다스 다중 필터
df_item_5 = df_item_5[df_item_5["store_nbr"].isin(c)]

In [ ]:
df_item_5

In [ ]:
cf.isThereNoneData(df_item_5)

In [ ]:
new_columns = list(weather.columns) + ["units"]
new_columns

In [ ]:
df_item_5_reg = df_item_5[new_columns]
df_item_5_reg

In [ ]:
cf.isThereNoneData(df_item_5_reg)

In [ ]:
cf.remove_columns(df_item_5_reg, ["depart","codesum","station_nbr","sunrise","sunset","date"])

In [ ]:
df_item_5_reg.head()

In [ ]:
df_item_5_reg = cf.remove_m_row(df_item_5_reg, df_item_5_reg.columns)

In [ ]:
df_item_5_reg.head()

In [ ]:
y = ["units"]
x = list(df_item_5_reg.columns)[:-1]

In [ ]:
x

In [ ]:
dfx0 = df_item_5_reg[x]
dfy = df_item_5_reg["units"]
dfX = sm.add_constant(dfx0) 

In [ ]:
dfX = sm.add_constant(dfx0) 